<a href="https://colab.research.google.com/github/andriasleburu/bolt-sitelegacy/blob/main/translator_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.9 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Thimbukushu_corpus.docx to Thimbukushu_corpus.docx


In [ ]:
import docx
import io

# The 'uploaded' dictionary contains the file content.
# We assume the user wants to process the 'Thimbukushu_corpus.docx' file.
docx_filename = "Thimbukushu_corpus.docx"

if docx_filename in uploaded:
    # Get the binary content of the uploaded file
    docx_content = uploaded[docx_filename]

    # Save the content to a file in the Colab environment
    with open(docx_filename, "wb") as f:
        f.write(docx_content)

    print(f"File '{docx_filename}' saved to /content/")

    # Now open the .docx file using the python-docx library
    document = docx.Document(docx_filename)

    # Example: Print the first paragraph of the document
    if document.paragraphs:
        print("\nFirst paragraph of the document:")
        print(document.paragraphs[0].text)
    else:
        print("\nDocument has no paragraphs.")
else:
    print(f"Error: File '{docx_filename}' not found in uploaded files.")

File 'Thimbukushu_corpus.docx' saved to /content/

First paragraph of the document:
{"src_lang": "eng_Latn", "tgt_lang": "umb_Latn", "src": "In the beginning was the Word.", "tgt": "Pakutanga dha karire N̂ando."}


In [ ]:
from docx import Document
import json
import re

# Load your document
doc = Document("Thimbukushu_corpus.docx")

raw_lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]

json_records = []

for line in raw_lines:
    # Normalize quotes
    line_clean = (
        line.replace("“", '"')
            .replace("”", '"')
            .replace("’", "'")
            .replace("‚", "'")
            .replace("„", '"')
    )

    # Try to detect a JSON object inside the line
    match = re.search(r'\{.*\}', line_clean)

    if match:
        json_text = match.group(0)
        try:
            obj = json.loads(json_text)

            # Only keep valid translation pairs
            if "src" in obj and "tgt" in obj:
                json_records.append({
                    "src_lang": "eng_Latn",
                    "tgt_lang": "umb_Latn",
                    "src": obj["src"],
                    "tgt": obj["tgt"]
                })
        except:
            pass


In [ ]:
output_path = "thimbukushu_corpus.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for rec in json_records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("Saved:", output_path)
print("Total valid records:", len(json_records))


Saved: thimbukushu_corpus.jsonl
Total valid records: 307


In [ ]:
files.download("thimbukushu_corpus.jsonl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
{"src_lang":"eng_Latn","tgt_lang":"umb_Latn","src":"Year.","tgt":"Mwáka."}


{'src_lang': 'eng_Latn',
 'tgt_lang': 'umb_Latn',
 'src': 'Year.',
 'tgt': 'Mwáka.'}

In [ ]:
{"src_lang": "eng_Latn", "tgt_lang": "umb_Latn", "src": "Year.", "tgt": "Mwaka."}


{'src_lang': 'eng_Latn',
 'tgt_lang': 'umb_Latn',
 'src': 'Year.',
 'tgt': 'Mwaka.'}

In [ ]:
DATA_PATH = "/content/thimbukushu_corpus.jsonl"  # Change if needed

with open(DATA_PATH, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

print("Total pairs:", len(data))


Total pairs: 307


In [ ]:
import random
random.shuffle(data)

train_split = 0.8
valid_split = 0.1

n = len(data)
train_data = data[:int(n*train_split)]
valid_data = data[int(n*train_split):int(n*(train_split+valid_split))]
test_data  = data[int(n*(train_split+valid_split)):]

ds = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(valid_data),
    "test": Dataset.from_list(test_data),
})

ds


DatasetDict({
    train: Dataset({
        features: ['src_lang', 'tgt_lang', 'src', 'tgt'],
        num_rows: 245
    })
    validation: Dataset({
        features: ['src_lang', 'tgt_lang', 'src', 'tgt'],
        num_rows: 31
    })
    test: Dataset({
        features: ['src_lang', 'tgt_lang', 'src', 'tgt'],
        num_rows: 31
    })
})

In [ ]:
MODEL_NAME = "facebook/nllb-200-distilled-600M"

SRC_LANG = "eng_Latn"
TGT_LANG = "umb_Latn"   # Placeholder for Thimbukushu

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, src_lang=SRC_LANG, tgt_lang=TGT_LANG)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
max_input_length = 128
max_target_length = 128

def preprocess(batch):
    model_inputs = tokenizer(
        batch["src"],
        max_length=max_input_length,
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["tgt"],
            max_length=max_target_length,
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = ds.map(preprocess, batched=True, remove_columns=ds["train"].column_names)


Map:   0%|          | 0/245 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,179,648 || all params: 616,253,440 || trainable%: 0.1914


In [ ]:
output_dir = "nllb_thimbukushu_lora"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    num_train_epochs=3,
    eval_strategy="epoch", # Changed from evaluation_strategy
    logging_steps=50,
    save_strategy="epoch",
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,  # will enable later
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
)

trainer.train()
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


/tmp/ipython-input-2258137674.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: andriasleburu (andriasleburu-mudhimu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,6.566700,6.601857
2,6.339300,6.355884
3,6.290900,6.258154


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


('nllb_thimbukushu_lora/tokenizer_config.json',
 'nllb_thimbukushu_lora/special_tokens_map.json',
 'nllb_thimbukushu_lora/sentencepiece.bpe.model',
 'nllb_thimbukushu_lora/added_tokens.json',
 'nllb_thimbukushu_lora/tokenizer.json')

In [ ]:
metric = evaluate.load("sacrebleu")

def translate(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

preds, refs = [], []

for ex in ds["test"]:
    pred = translate(ex["src"])
    preds.append(pred)
    refs.append([ex["tgt"]])

print("Sample prediction:", preds[0])
print("Reference:", refs[0])

bleu = metric.compute(predictions=preds, references=refs)
bleu


Sample prediction: Τρίτη.
Reference: ['Ghuwadi.']


{'score': 2.225287173746553,
 'counts': [29, 0, 0, 0],
 'totals': [90, 59, 29, 12],
 'precisions': [32.22222222222222,
  0.847457627118644,
  0.8620689655172413,
  1.0416666666666667],
 'bp': 1.0,
 'sys_len': 90,
 'ref_len': 74}

In [ ]:
from huggingface_hub import login
login()  # paste your HF token here

In [ ]:
repo_id = "andriasleburu/thimbukushu-translator-small"

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:  93%|#########2| 4.39MB / 4.74MB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...w/sentencepiece.bpe.model:  98%|#########7| 4.74MB / 4.85MB            

  ...mpee1q8osw/tokenizer.json:  22%|##1       | 6.97MB / 32.2MB            

CommitInfo(commit_url='https://huggingface.co/andriasleburu/thimbukushu-translator-small/commit/8d440cc8b22c01871f848a0ac7945cc8d780e0e6', commit_message='Upload tokenizer', commit_description='', oid='8d440cc8b22c01871f848a0ac7945cc8d780e0e6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/andriasleburu/thimbukushu-translator-small', endpoint='https://huggingface.co', repo_type='model', repo_id='andriasleburu/thimbukushu-translator-small'), pr_revision=None, pr_num=None)